In [1]:
from ctypes import *

runtime_ds = CDLL("../target/release/libruntime_datastructure.so")

In [2]:
class C_Label(Structure):
    _fields_ = [("x", c_double),
                ("y", c_double),
                ("t", c_double),
                ("osm_id", c_long),
                ("prio", c_int),
                ("label", c_char_p)]
    
class C_Result(Structure):
    _fields_ = [("size", c_uint64),
                ("data", POINTER(C_Label))]
    
class Label:
    def __init__(self, l):
        self.x = l.x
        self.y = l.y
        self.t = l.t
        
        self.osm_id = l.osm_id
        self.prio = l.prio
        self.label = l.label.decode("utf-8")
        
    def to_string(self):
        return "Label #{}: '{}' at (x: {}, y:{}) with prio {} has t = {}".format(self.osm_id, self.label, self.x, self.y, self.prio, self.t)

In [3]:
# init
runtime_ds.init.argtypes = [c_char_p]
runtime_ds.init.restype = c_void_p

# is_good
runtime_ds.is_good.argtypes = [c_void_p]
runtime_ds.is_good.restype = c_bool

#get_data
runtime_ds.get_data.argtypes = [c_void_p, c_double, c_double, c_double, c_double, c_double]
runtime_ds.get_data.restype = C_Result

In [4]:
def to_c_string(s):
    return create_string_buffer(s.encode('utf-8'))

In [5]:
ds = runtime_ds.init(to_c_string("../resources/baden-wuerttemberg-latest.osm.pbf.ce"))

print("Initialisieren der Datenstuktur war erfolgreich: {}".format(runtime_ds.is_good(ds)))

Initialisieren der Datenstuktur war erfolgreich: True


In [6]:
min_t = 0
min_lon = 9.16402
max_lon = 9.24161
min_lat = 49.11342
max_lat = 49.16971


res = runtime_ds.get_data(ds, min_t, min_lon, max_lon, min_lat, max_lat)

l = list()
for i in range(res.size):
    l.append(Label(res.data[i]))

print("Found {} matching items:\n".format(res.size))
for e in l:
    print("{}".format(e.to_string()))

Found 24 matching items:

Label #26769661: 'Heilbronn' at (x: 9.218655, y:49.142291) with prio 14075 has t = 3.2690108033729004
Label #1618716160: 'Horkheim' at (x: 9.1676575, y:49.116147700000006) with prio 9801 has t = 0.5350372850345787
Label #1556687999: 'Frankenbach' at (x: 9.1717549, y:49.159423800000006) with prio 10022 has t = 0.3950841200171466
Label #2626294843: 'Haselter' at (x: 9.1758971, y:49.13852180000001) with prio 8938 has t = 0.19055895000850182
Label #2626294838: 'Gewerbegebiet\nBöckingen West' at (x: 9.1643681, y:49.1413585) with prio 8937 has t = 0.12355901164664317
Label #2626882442: 'Schanz' at (x: 9.180747, y:49.1457661) with prio 8957 has t = 0.27867669649156473
Label #1564935149: 'Sontheim' at (x: 9.1884022, y:49.118011200000005) with prio 9777 has t = 0.2648671445641906
Label #2626882430: 'Alt-Böckingen' at (x: 9.189984, y:49.1328405) with prio 8950 has t = 0.03408735157879569
Label #1547719713: 'Böckingen' at (x: 9.1932571, y:49.133695) with prio 10045 has t

In [7]:
res.size

24